### Lab Assignment: Python Warmup and Logfile Analytics 

### University of Virginia
### DS 7200: Distributed Computing
### Last Updated: August 20, 2023

---

This lab consists of two parts. Part 1 is the Python warmup. Part 2 is the logfile analytics in Spark.  
Answer the questions in this assignment, showing all code and solutions.

**Total points: 15**

---

### Part 1: Python Warmup

1) (1 PT) Rename this notebook to JupyterTutorial_[your_initials], where you will enter your initials in place of [your_initials].

2) (1 PT) In the cell below, enter a list of data science topics you find interesting.  Use the markdown style (you will need to change the style from the Code style).

- handling of missing data
- the intersection of explainability and predictability in machine learning models
- convolutional neural networks

3) (1 PT) In the cell below, enter the following Python list:

some_vals = [-1, 6, 12, 34]  

You will use the Code style, run the cell, and print the list.

In [2]:
some_vals = [-1, 6, 12, 34]

4) (1 PT) Use a list comprehension to return a filtered list containing only the values greater than 6.  
Call this list *some_vals_filtered* and print it.

In [3]:
some_vals_filtered = [x for x in some_vals if x > 6]
print(some_vals_filtered)

[12, 34]


Next, a small pandas dataframe is constructed.

In [4]:
import pandas as pd

df = pd.DataFrame({'first_name': ['Andy','Crystal'],
                   'domain_facebook' : [1,1],
                   'domain_foursquare' : [0,0],
                   'age' : [20, 32]})
df

,first_name,domain_facebook,domain_foursquare,age
0,Andy,1,0,20
1,Crystal,1,0,32


5) (1 PT) In the cell below, write a list comprehension that returns the fields names in the dataframe `df` containing the string *domain*.  Run the cell to verify the correct result.

In [5]:
dm = [x for x in df.columns if x.startswith("domain")]
print(dm)

['domain_facebook', 'domain_foursquare']


6) (1 PT) Use the list comprehension from (5) to index into `df` and show the data for columns containing *domain*

In [7]:
df[dm]

,domain_facebook,domain_foursquare
0,1,0
1,1,0


7) (1 PT) In the cell below, print the *domain_facebook* column

In [8]:
df[dm[0]]

0    1
1    1
Name: domain_facebook, dtype: int64

8) (1 PT) In the cell below, print the row with index 1.

In [9]:
df[1:]

,first_name,domain_facebook,domain_foursquare,age
1,Crystal,1,0,32


9) (1 PT) Next, you will cube the *age* column of `df` and assign the result to a new column called *agecube*.

Specifically, call the `apply` method with a `lambda function` inside to cube the *age* column.  
Print the dataframe.

In [11]:
df['agecube'] = df.age.apply(lambda x: x**3)
print(df)

  first_name  domain_facebook  domain_foursquare  age  agecube
0       Andy                1                  0   20     8000
1    Crystal                1                  0   32    32768


10) (1 PT) Given the list of strings below, form one string, placing semicolons between each word.  It should look like this:  

`'the;quick;brown;fox'`

Print the resulting string.

In [12]:
some_list = ['the','quick','brown','fox']

In [13]:
some_list_string = ";".join(some_list)
print(some_list_string)

the;quick;brown;fox


### Part 2: Logfile Analytics

Import modules for Spark Session and regex 

In [14]:
from pyspark.sql import SparkSession
import re

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

24/09/05 16:57:01 WARN Utils: Your hostname, Eileanors-Laptop.local resolves to a loopback address: 127.0.0.1; using 172.25.130.134 instead (on interface en0)
24/09/05 16:57:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/05 16:57:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


11) (1 PT) Read in the logfile.txt data

In [15]:
data_filename = 'logfile.txt'
lines = sc.textFile(data_filename)

12) (1 PT) Count the number of rows of data

In [16]:
lines.count()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py:154: DeprecationWarning: This process (pid=42061) is multi-threaded, use of fork() may lead to deadlocks in the child.


360

13) (1 PT) Show the first 5 lines

In [17]:
lines.collect()[:5]

[' 01 ',
 '03/22 08:51:01 INFO   :.main: *************** RSVP Agent started ***************',
 ' 02 ',
 '03/22 08:51:01 INFO   :...locate_configFile: Specified configuration file: /u/user10/rsvpd1.conf',
 '03/22 08:51:01 INFO   :.main: Using log level 511']

14) (1 PT) Show all lines containing the word WARNING and write code to count them

In [18]:
spark_lines = lines.filter(lambda x: "WARNING" in x)
spark_lines.collect()

['03/22 08:51:06 WARNING:.....mailslot_create: setsockopt(MCAST_ADD) failed - EDC8116I Address not available.',
 '03/22 08:51:06 WARNING:.....mailslot_create: setsockopt(MCAST_ADD) failed - EDC8116I Address not available.',
 '03/22 08:51:06 WARNING:.....mailslot_create: setsockopt(MCAST_ADD) failed - EDC8116I Address not available.',
 '03/22 08:51:06 WARNING:.....mailslot_create: setsockopt(MCAST_ADD) failed - EDC8116I Address not available.']

In [19]:
linecounts = lines.filter(lambda x: "WARNING" in x) \
            .map(lambda x: (x, 1)) \
            .reduceByKey(lambda x,y:x+y) \
            .map(lambda x:(x[1],x[0])) \
            .sortByKey(False)

linecounts.collect() #4 instances of the same WARNING

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:65: UserWarning: Please install psutil to have better support with spilling
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:65: UserWarning: Please install psutil to have better support with spilling


[(4,
  '03/22 08:51:06 WARNING:.....mailslot_create: setsockopt(MCAST_ADD) failed - EDC8116I Address not available.')]

15) (1 PT) Write a word count program to count the number of each of these log levels:  

terms = 'WARNING|INFO|EVENT|PROTERR|TRACE'

In [38]:
words = lines.flatMap(lambda x: x.split('.'|' '))

wordcounts = words.filter(lambda x: 'INFO' in x)

wordcounts.collect()


24/09/05 17:07:32 ERROR Executor: Exception in task 0.0 in stage 26.0 (TID 44)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)
        

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 26.0 failed 1 times, most recent failure: Lost task 1.0 in stage 26.0 (TID 45) (172.25.130.134 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
         ^^^^^^^^^^^^